In [2]:
import pandas as pd
import numpy as np

# Define the 4D array for the group_a
table_A = np.array([
    [
        [[1, 2], [2, 2], [2, 3], [3, 3]],
        [[2, 2], [2, 2], [3, 3], [3, 3]],
        [[2, 3], [3, 3], [3, 3], [4, 4]],
    ],
    [
        [[2, 3], [3, 3], [3, 4], [4, 4]],
        [[3, 3], [3, 3], [3, 4], [4, 4]],
        [[3, 4], [4, 4], [4, 4], [5, 5]],
    ],
    [
        [[3, 3], [4, 4], [4, 4], [5, 5]],
        [[3, 4], [4, 4], [4, 4], [5, 5]],
        [[4, 4], [4, 5], [5, 5], [5, 5]],
    ],
    [
        [[4, 4], [4, 4], [4, 5], [5, 5]],
        [[4, 4], [4, 4], [4, 5], [5, 6]],
        [[3, 4], [5, 5], [5, 6], [6, 6]],
    ],
    [
        [[1, 5], [5, 6], [6, 6], [6, 6]],
        [[2, 5], [6, 6], [6, 7], [7, 7]],
        [[3, 6], [6, 7], [7, 7], [7, 8]],
    ],
    [
        [[1, 7], [7, 8], [8, 8], [8, 9]],
        [[2, 8], [8, 9], [9, 9], [9, 9]],
        [[3, 9], [9, 9], [9, 9], [9, 9]],
    ],
])

# Define the 4D array for the group_b
table_B = np.array([
    [[1, 3], [2, 3], [3, 4], [5, 5], [6, 6], [7, 7]],
    [[2, 3], [2, 3], [4, 5], [5, 5], [6, 7], [7, 7]],
    [[3, 3], [3, 4], [4, 5], [5, 6], [6, 7], [7, 7]],
    [[5, 5], [5, 6], [6, 7], [7, 7], [7, 7], [8, 8]],
    [[7, 7], [7, 7], [7, 8], [8, 8], [8, 8], [8, 8]],
    [[8, 8], [8, 8], [8, 8], [8, 9], [9, 9], [9, 9]],
])

table_C = np.array([
    [1, 2, 3, 4, 5, 6, 7, 7, 7, 7, 7],
    [2, 2, 3, 4, 4, 5, 5, 7, 7, 7, 7],
    [3, 3, 3, 4, 4, 5, 6, 7, 7, 7, 7],
    [3, 3, 3, 4, 5, 6, 6, 7, 7, 7, 7],
    [4, 4, 4, 5, 6, 7, 7, 7, 7, 7, 7],
    [4, 4, 5, 6, 6, 7, 7, 7, 7, 7, 7],
    [5, 5, 6, 6, 7, 7, 7, 7, 7, 7, 7],
    [5, 5, 6, 7, 7, 7, 7, 7, 7, 7, 7],
    [7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7],
    [7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7],
    [7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7],
])

# Load data from CSV
file_path = "1001_Right _10minAng.csv"  # Replace with your actual file path
data = pd.read_csv(file_path)

# Initialize an empty list to store results
results = []

# Iterate through each row
for _, row in data.iterrows():
    # Step 2: Get inputs for Group A
    upper_arm = int(row['upper_arm(1-6)'])
    lower_arm = int(row['lower_arm(1-3)'])
    wrist = int(row['wrist(1-4)'])
    wrist_twist = int(row['wrist_twist(1-2)'])
    muscle_use_a = int(row['muscle_use_a(0-1)'])
    load_a = int(row['load_a(0-3)'])

    # Lookup Group A score
    group_a = table_A[upper_arm - 1][lower_arm - 1][wrist - 1][wrist_twist - 1]
    new_a = group_a + muscle_use_a + load_a

    # Step 3: Get inputs for Group B
    neck = int(row['neck(1-6)'])
    trunk = int(row['trunk(1-6)'])
    legs = int(row['legs(1-2)'])
    muscle_use_b = int(row['muscle_use_b(0-1)'])
    load_b = int(row['load_b(0-3)'])

    # Lookup Group B score
    group_b = table_B[neck - 1][trunk - 1][legs - 1]
    new_b = group_b + muscle_use_b + load_b

    # Step 4: Lookup Final RULA Score
    rula_score = table_C[new_a - 1][new_b - 1]

    # Append results
    results.append({
        'Row_Index': _ + 1,  # Track the index of the row
        'Group_A': group_a,
        'Arm_Wrist_Score': new_a,
        'Group_B': group_b,
        'Neck_Trunk_Legs_Score': new_b,
        'RULA_Score': rula_score
    })

# Convert results to DataFrame
results_df = pd.DataFrame(results)
results_df.to_csv("1001_Right_rula_results.csv", index=False)
print("Intermediate results saved to '1001_Right_rula_results.csv'.")

# Calculate Weighted RULA Score
weighted_results = results_df['RULA_Score'].value_counts().reset_index()
weighted_results.columns = ['RULA_Score', 'Frequency']
weighted_results['Weighted_Sum'] = weighted_results['RULA_Score'] * weighted_results['Frequency']
total_weighted_sum = weighted_results['Weighted_Sum'].sum()
total_frequency = weighted_results['Frequency'].sum()
weighted_rula_score = total_weighted_sum / total_frequency if total_frequency > 0 else 0

# Save Weighted RULA Score to a CSV
weighted_results['Weighted_RULA_Score'] = weighted_rula_score
weighted_results.to_csv("1001_Right_weighted_rula_results.csv", index=False)
print("Weighted RULA results saved to '1001_Right_weighted_rula_results.csv'.")


Intermediate results saved to '1001_Right_rula_results.csv'.
Weighted RULA results saved to '1001_Right_weighted_rula_results.csv'.
